In [1]:
import pandas as pd
import re

In [2]:
#start data file
df_all=pd.read_csv('PerseusNormalizedData.csv')

In [4]:
#generate whole phosphoproteomic dataset with site information
MajorProtein=[]
for i in range(0,df_all['Proteins'].shape[0]):
    MajorProtein.append(re.search("^[^;]+", df_all['Proteins'][i]).group())
df_all['MajorProtein']=MajorProtein

df_all['Gene names']=df_all['Gene names'].fillna(df_all['MajorProtein'])

Position=[]
for i in range(0,df_all['Positions within proteins'].shape[0]):
    Position.append(re.search("^[^;]+", df_all['Positions within proteins'][i]).group())
df_all['Position']=Position

sit=[]
proba=[]
for i in range(0,df_all['Phospho (STY) Probabilities'].shape[0]):
    spl=list()
    for match in re.finditer("[STY]\(.{1,5}\)", df_all['Phospho (STY) Probabilities'][i]): 
        split=(re.split('[()]',match.group(0)))
        list(split)
        spl.append(split)
    df_spl=pd.DataFrame(spl)
    df_spl_sort=df_spl.sort_values(1,ascending=False)
    sit.append(df_spl_sort.iloc[0,0])
    proba.append(df_spl_sort.iloc[0,1])
df_all['Phospho']=sit
df_all['Prob']=proba

Gene=[]
for i in range(0,df_all['Gene names'].shape[0]):
    Gene.append(re.search("^[^;]+", df_all['Gene names'][i]).group())
df_all['Gene']=Gene

df_all['ProteinAndSite']=df_all['Gene']+'_'+df_all['Phospho']+df_all['Position']
#df_all.to_csv('YD1456_Phos_Report_PhosphositeCombine.csv')

In [5]:
# Create a new column to annotate duplicates
df_all['dup_count'] = df_all.groupby('ProteinAndSite').cumcount()
df_all["Uniquephosite"] = df_all.apply(
    lambda row: f"{row['ProteinAndSite']}_iso{row['dup_count']}" if row['dup_count']>0 else row['ProteinAndSite'],
    axis=1
)
df_all.drop(columns='dup_count', inplace=True)

In [6]:
#confirm no replicates in the column
df_all["Uniquephosite"].duplicated().any() 

False

In [7]:
#subset dataframe
df=df_all.set_index('Uniquephosite').iloc[:,:16]

In [8]:
#end data file
#df.to_csv("Uniquephosite.csv")